In [0]:
%pip install azure-eventhub
dbutils.library.restartPython()

In [0]:
# Event Hub Configuration
eventhub_namespace = "evhns-natraining.servicebus.windows.net"
eventhub_name = "evh-natraining-biju"
keyvault_scope = "dbx-ss-kv-natraining-2"
secret_name = "evh-natraining-read-write"
shared_access_key_name = "SharedAccessKeyToSendAndListen"

try:
    # The actual Shared Access Key value is retrieved and stored in 'secret_value'
    secret_value = dbutils.secrets.get(
        scope=keyvault_scope,
        key=secret_name
    )
    print("✓ Successfully retrieved secret from Key Vault")
    print(f"  - Secret name: {secret_name}")
    print(f"  - Scope: {keyvault_scope}")
except Exception as e:
    print(f"✗ Error retrieving secret: {str(e)}")
    raise


# Build connection string
# FIX: Replaced 'shared_access_key' with the actual secret value variable: 'secret_value'
connection_string = (
    f"Endpoint=sb://{eventhub_namespace}/;"
    f"SharedAccessKeyName={shared_access_key_name};"
    f"SharedAccessKey={secret_value};" 
    f"EntityPath={eventhub_name}"
)

print("\n--- Generated Connection String ---")
# Use 'dbutils.secrets.mask()' to safely print the connection string without exposing the key
print(connection_string.replace(secret_value, '***'))

In [0]:
import json
import time
from datetime import datetime
from azure.eventhub import EventHubProducerClient, EventData
import random

# COMMAND ----------

# Product catalog
products = [
    {"product_id": "P001", "name": "Laptop", "category": "Electronics", "brand": "Dell", "base_price": 1200},
    {"product_id": "P002", "name": "Smartphone", "category": "Electronics", "brand": "Samsung", "base_price": 800},
    {"product_id": "P003", "name": "Headphones", "category": "Accessories", "brand": "Sony", "base_price": 150},
    {"product_id": "P004", "name": "Keyboard", "category": "Accessories", "brand": "Logitech", "base_price": 80},
    {"product_id": "P005", "name": "Monitor", "category": "Electronics", "brand": "LG", "base_price": 400},
    {"product_id": "P006", "name": "Mouse", "category": "Accessories", "brand": "Logitech", "base_price": 50},
    {"product_id": "P007", "name": "Tablet", "category": "Electronics", "brand": "Apple", "base_price": 600},
    {"product_id": "P008", "name": "Webcam", "category": "Accessories", "brand": "Logitech", "base_price": 100},
    {"product_id": "P009", "name": "Smartwatch", "category": "Electronics", "brand": "Apple", "base_price": 350},
    {"product_id": "P010", "name": "USB Cable", "category": "Accessories", "brand": "Generic", "base_price": 15}
]

customer_names = ["John Smith", "Jane Doe", "Bob Johnson", "Alice Williams", "Charlie Brown", 
                  "Diana Davis", "Eve Martinez", "Frank Wilson", "Grace Lee", "Henry Taylor"]

locations = ["New York", "Los Angeles", "Chicago", "Houston", "Phoenix", 
             "Philadelphia", "San Antonio", "San Diego", "Dallas", "San Jose"]

# COMMAND ----------

def generate_enriched_order(order_id):
    """Generate enriched order event with product details embedded"""
    
    customer_id = f"C{random.randint(1000, 1050):04d}"
    product = random.choice(products)
    quantity = random.randint(1, 5)
    discount = random.choice([0, 0.05, 0.10, 0.15])
    price = product["base_price"] * (1 - discount)
    
    event = {
        # Order Information
        "order_id": f"ORD{order_id:06d}",
        "customer_id": customer_id,
        "customer_name": random.choice(customer_names),
        "location": random.choice(locations),
        "order_status": random.choice(["pending", "confirmed", "shipped"]),
        "payment_method": random.choice(["credit_card", "debit_card", "paypal"]),
        "quantity": quantity,
        "discount_pct": discount,
        "total_amount": round(price * quantity, 2),
        "order_timestamp": datetime.utcnow().isoformat(),
        
        # Product Information (embedded)
        "product_id": product["product_id"],
        "product_name": product["name"],
        "category": product["category"],
        "brand": product["brand"],
        "base_price": product["base_price"],
        "unit_price": round(price, 2)
    }
    
    return event

# COMMAND ----------

def send_events():
    """Send enriched order events to Event Hub"""
    
    producer = EventHubProducerClient.from_connection_string(
        conn_str=connection_string.replace(f";EntityPath={eventhub_name}", ""),
        eventhub_name=eventhub_name
    )
    
    print("Starting event generation for 60 seconds...")
    print("Each event contains BOTH order and product information\n")
    
    start_time = time.time()
    order_count = 0
    
    try:
        while time.time() - start_time < 60:
            # Send 5 orders per second
            batch = producer.create_batch()
            for i in range(5):
                event = generate_enriched_order(order_count)
                batch.add(EventData(json.dumps(event)))
                order_count += 1
            
            producer.send_batch(batch)
            
            # Progress update
            if order_count % 50 == 0:
                elapsed = time.time() - start_time
                print(f"[{elapsed:.0f}s] Generated: {order_count} enriched orders")
            
            time.sleep(1)
    
    finally:
        producer.close()
    
    print(f"\n✓ Generation complete!")
    print(f"✓ Total enriched orders: {order_count}")
    print(f"✓ Each order contains order details + product details")

# COMMAND ----------

# Run the generator
send_events()